In [165]:
from aocd.models import Puzzle
import math
import pprint

In [166]:
puzzle = Puzzle(2019, 10)
puzzle_input = puzzle.input_data.split("\n")

pprint.pprint(puzzle_input)

['.###..#######..####..##...#',
 '########.#.###...###.#....#',
 '###..#...#######...#..####.',
 '.##.#.....#....##.#.#.....#',
 '###.#######.###..##......#.',
 '#..###..###.##.#.#####....#',
 '#.##..###....#####...##.##.',
 '####.##..#...#####.#..###.#',
 '#..#....####.####.###.#.###',
 '#..#..#....###...#####..#..',
 '##...####.######....#.####.',
 '####.##...###.####..##....#',
 '#.#..#.###.#.##.####..#...#',
 '..##..##....#.#..##..#.#..#',
 '##.##.#..######.#..#..####.',
 '#.....#####.##........#####',
 '###.#.#######..#.#.##..#..#',
 '###...#..#.#..##.##..#####.',
 '.##.#..#...#####.###.##.##.',
 '...#.#.######.#####.#.####.',
 '#..##..###...###.#.#..#.#.#',
 '.#..#.#......#.###...###..#',
 '#.##.#.#..#.#......#..#..##',
 '.##.##.##.#...##.##.##.#..#',
 '#.###.#.#...##..#####.###.#',
 '#.####.#..#.#.##.######.#..',
 '.#.#####.##...#...#.##...#.']


In [167]:
def create_matrix(lst):
    # transpose input data to make traversial easier
    return ["".join(s) for s in zip(*lst)]      

In [168]:
puzzle_matrix = create_matrix(puzzle_input)

# TESTS:

# (3, 4) => 8 -> works
# puzzle_matrix = create_matrix([".#..#", ".....", "#####", "....#", "...##"])

# (5, 8) => 33 -> works
# puzzle_matrix = create_matrix(["......#.#.", "#..#.#....", "..#######.", ".#.#.###..", ".#..#.....", "..#....#.#", "#..#....#.", ".##.#..###", "##...#..#.", ".#....####"])

# (1, 2) => 35 -> works
# puzzle_matrix = create_matrix(["#.#...#.#.", ".###....#.", ".#....#...", "##.#.#.#.#", "....#.#.#.", ".##..###.#", "..#...##..", "..##....##", "......#...", ".####.###."])

# (6, 3) => 41 -> works
# puzzle_matrix = create_matrix([".#..#..###", "####.###.#", "....###.#.", "..###.##.#", "##.##.#.#.", "....###..#", "..#.#..#.#", "#..#.#.###", ".##...##.#", ".....#.#.."])

# (11, 13) => 210 -> works
# puzzle_matrix = create_matrix([".#..##.###...#######", "##.############..##.", ".#.######.########.#", ".###.#######.####.#.", "#####.##.#.##.###.##", "..#####..#.#########", "####################", "#.####....###.#.#.##", "##.#################", "#####.##.###..####..", "..######..##.#######", "####.##.####...##..#", ".#####..#.######.###", "##...#.##########...", "#.##########.#######", ".####.#.###.###.#.##", "....##.##.###..#####", ".#.#.###########.###", "#.#.#.#####.####.###", "###.##.####.##.#..##"])

In [169]:
def calculate_distance_vector(vector1, vector2):
    return tuple([x[1] - x[0] for x in list(zip(vector1, vector2))])

In [170]:
def standardize_vector(vector):
    vector_length = math.sqrt(sum([x**2 for x in vector]))
    vector = tuple([round(x / vector_length, 5) for x in vector])
    return vector

In [171]:
def check_if_collinear(vector1, vector2):
    verdict = standardize_vector(vector1) == standardize_vector(vector2)
    return verdict

In [172]:
# this function counts all asteroids that are within LoS of another asteroid
def count_asteroids(coords, asteroid_map):
    
    asteroids_discovered = 0
    already_discovered_vectors = []
        
    # Scan over each element in the map (going from top left to bottom right)
    for row in range(len(asteroid_map)):
        for col in range(len(asteroid_map[0])):
            if (col, row) != coords and asteroid_map[col][row] == "#":
                # there is an asteroid here
                asteroid_coords = (col, row)
                
                # calculate distance vector
                distance_vector = calculate_distance_vector(coords, asteroid_coords)
                
                # if distance vector happens to be collinear with the vector of another asteroid you found, ignore
                non_collinear_vector_found = True
                for vector in already_discovered_vectors:
                    if (check_if_collinear(vector, distance_vector)):
                        non_collinear_vector_found = False
                        
                if not non_collinear_vector_found:
                    continue
                
                # else add this vector to list of vectors you have already scanned, increment asteroids in LoS by 1
                else:
                    already_discovered_vectors.append(distance_vector)
                    asteroids_discovered += 1
                    
    return asteroids_discovered, already_discovered_vectors

In [173]:
max_asteroids_in_line_of_sight = 0
coords_of_asteroid_with_best_line_of_sight = (0, 0)

los_asteroid_vectors = [] # line of sight asteroid vectors

# column is X, row is Y
for row in range(len(puzzle_matrix)):
    for col in range(len(puzzle_matrix[0])):
        if puzzle_matrix[col][row] == "#": # found an asteroid, now let's scan!
            
            coords_tuple = (col, row)
                       
            asteroids_in_line_of_sight = count_asteroids(coords_tuple, puzzle_matrix)[0]
                        
            # update with asteroid with new best vantage point
            if asteroids_in_line_of_sight > max_asteroids_in_line_of_sight:
                max_asteroids_in_line_of_sight = asteroids_in_line_of_sight
                coords_of_asteroid_with_best_line_of_sight = coords_tuple
                los_asteroid_vectors = count_asteroids(coords_tuple, puzzle_matrix)[1]
                
print(max_asteroids_in_line_of_sight, coords_of_asteroid_with_best_line_of_sight, los_asteroid_vectors)

296 (17, 23) [(-16, -23), (-15, -23), (-14, -23), (-11, -23), (-10, -23), (-9, -23), (-8, -23), (-7, -23), (-6, -23), (-5, -23), (-2, -23), (-1, -23), (0, -23), (1, -23), (4, -23), (5, -23), (9, -23), (-17, -22), (-16, -22), (-15, -22), (-14, -22), (-13, -22), (-12, -22), (-11, -22), (-10, -22), (-8, -22), (-6, -22), (-5, -22), (-4, -22), (1, -22), (2, -22), (4, -22), (9, -22), (-17, -21), (-16, -21), (-15, -21), (-12, -21), (-8, -21), (-7, -21), (-6, -21), (-5, -21), (-4, -21), (-3, -21), (-2, -21), (2, -21), (5, -21), (6, -21), (7, -21), (8, -21), (-16, -20), (-15, -20), (-13, -20), (-7, -20), (-2, -20), (-1, -20), (1, -20), (3, -20), (9, -20), (-17, -19), (-16, -19), (-15, -19), (-13, -19), (-12, -19), (-11, -19), (-10, -19), (-9, -19), (-8, -19), (-7, -19), (-5, -19), (-4, -19), (-3, -19), (1, -19), (8, -19), (-17, -18), (-14, -18), (-13, -18), (-12, -18), (-8, -18), (-7, -18), (-5, -18), (-4, -18), (-2, -18), (1, -18), (2, -18), (3, -18), (4, -18), (9, -18), (-17, -17), (-15, -17)

In [185]:
# answer was correct, now onto part B

los_asteroid_vectors = [(-16, -23), (-15, -23), (-14, -23), (-11, -23), (-10, -23), (-9, -23), (-8, -23), (-7, -23), (-6, -23), (-5, -23), (-2, -23), (-1, -23), (0, -23), (1, -23), (4, -23), (5, -23), (9, -23), (-17, -22), (-16, -22), (-15, -22), (-14, -22), (-13, -22), (-12, -22), (-11, -22), (-10, -22), (-8, -22), (-6, -22), (-5, -22), (-4, -22), (1, -22), (2, -22), (4, -22), (9, -22), (-17, -21), (-16, -21), (-15, -21), (-12, -21), (-8, -21), (-7, -21), (-6, -21), (-5, -21), (-4, -21), (-3, -21), (-2, -21), (2, -21), (5, -21), (6, -21), (7, -21), (8, -21), (-16, -20), (-15, -20), (-13, -20), (-7, -20), (-2, -20), (-1, -20), (1, -20), (3, -20), (9, -20), (-17, -19), (-16, -19), (-15, -19), (-13, -19), (-12, -19), (-11, -19), (-10, -19), (-9, -19), (-8, -19), (-7, -19), (-5, -19), (-4, -19), (-3, -19), (1, -19), (8, -19), (-17, -18), (-14, -18), (-13, -18), (-12, -18), (-8, -18), (-7, -18), (-5, -18), (-4, -18), (-2, -18), (1, -18), (2, -18), (3, -18), (4, -18), (9, -18), (-17, -17), (-15, -17), (-14, -17), (-11, -17), (-10, -17), (-9, -17), (-4, -17), (-3, -17), (-2, -17), (-1, -17), (4, -17), (5, -17), (7, -17), (8, -17), (-17, -16), (-15, -16), (-14, -16), (-11, -16), (-4, -16), (-3, -16), (-2, -16), (-1, -16), (2, -16), (5, -16), (6, -16), (7, -16), (9, -16), (-17, -15), (-14, -15), (-9, -15), (-8, -15), (-7, -15), (-6, -15), (-4, -15), (-3, -15), (-2, -15), (-1, -15), (1, -15), (2, -15), (3, -15), (7, -15), (8, -15), (9, -15), (-17, -14), (-11, -14), (-6, -14), (-5, -14), (1, -14), (2, -14), (3, -14), (-17, -13), (-16, -13), (-12, -13), (-11, -13), (-10, -13), (-9, -13), (-7, -13), (-6, -13), (-5, -13), (-4, -13), (-3, -13), (-2, -13), (3, -13), (5, -13), (6, -13), (7, -13), (8, -13), (-17, -12), (-16, -12), (-15, -12), (-14, -12), (-11, -12), (-7, -12), (-5, -12), (-2, -12), (-1, -12), (3, -12), (9, -12), (-17, -11), (-15, -11), (-12, -11), (-10, -11), (-9, -11), (-1, -11), (5, -11), (9, -11), (-15, -10), (-14, -10), (-11, -10), (-3, -10), (1, -10), (4, -10), (9, -10), (-17, -9), (-16, -9), (-14, -9), (-13, -9), (-11, -9), (-8, -9), (-5, -9), (5, -9), (6, -9), (7, -9), (8, -9), (-17, -8), (-11, -8), (-9, -8), (-5, -8), (5, -8), (7, -8), (8, -8), (9, -8), (-17, -7), (-16, -7), (-15, -7), (-13, -7), (-11, -7), (-10, -7), (-9, -7), (-8, -7), (-6, -7), (3, -7), (6, -7), (9, -7), (-17, -6), (-16, -6), (-15, -6), (-11, -6), (5, -6), (7, -6), (8, -6), (-16, -5), (-15, -5), (-13, -5), (-10, -5), (-6, -5), (4, -5), (7, -5), (8, -5), (-14, -4), (-9, -4), (-7, -4), (5, -4), (6, -4), (7, -4), (8, -4), (-17, -3), (-14, -3), (-13, -3), (-10, -3), (5, -3), (7, -3), (9, -3), (-16, -2), (-13, -2), (-11, -2), (5, -2), (9, -2), (-17, -1), (-15, -1), (-14, -1), (-12, -1), (-10, -1), (-7, -1), (-5, -1), (5, -1), (8, -1), (9, -1), (-16, 0), (1, 0), (-17, 1), (-15, 1), (-14, 1), (-13, 1), (-11, 1), (-9, 1), (-5, 1), (-4, 1), (-1, 1), (0, 1), (1, 1), (2, 1), (3, 1), (5, 1), (6, 1), (7, 1), (9, 1), (-17, 2), (-15, 2), (-14, 2), (-13, 2), (-12, 2), (-7, 2), (-5, 2), (-3, 2), (1, 2), (3, 2), (5, 2), (7, 2), (-16, 3), (-14, 3), (-13, 3), (-11, 3), (-10, 3), (-8, 3), (-7, 3), (1, 3), (4, 3), (8, 3)]

"""
there are 4 quadrants in a coordinate system: Q1, Q2, Q3, Q4
Q1 - both coordinates are positive (up right)
Q2 - x is positive, y is negative (bottom right)
Q3 - both are negative (bottom left)
Q4 - x is negative, y is positive (top left)
"""


'\nthere are 4 quadrants in a coordinate system: Q1, Q2, Q3, Q4\nQ1 - both coordinates are positive (up right)\nQ2 - x is positive, y is negative (bottom right)\nQ3 - both are negative (bottom left)\nQ4 - x is negative, y is positive (top left)\n'

In [186]:
# this function iterates through every vector in the vector list and adds its radian to the vector as a third property
def add_radian(vectors):
    vector_list = vectors
    for ix, vector in enumerate(vector_list):
        try:
            degree = round(math.atan(vector[1]/vector[0]) * 180/math.pi, 6)
        except ZeroDivisionError:
            degree = 90.0
        vector_list[ix] = (vector[0], vector[1], degree)
    return vector_list

In [187]:
# step 1: filter distance vectors to their corresponding quadrant

# for this we will have to invert the y coordinates as we now need the original representation of (0, 0) instead of 
# (0, 0) being top left

los_asteroid_vectors = list(map(lambda x: (x[0], x[1] * -1) ,los_asteroid_vectors))


vectors_q1 = list(filter(lambda x: x[0] >= 0 and x[1] >= 0, los_asteroid_vectors[:]))
vectors_q2 = list(filter(lambda x: x[0] >= 0 and x[1] < 0, los_asteroid_vectors[:]))
vectors_q3 = list(filter(lambda x: x[0] < 0 and x[1] < 0, los_asteroid_vectors[:]))
vectors_q4 = list(filter(lambda x: x[0] < 0 and x[1] >= 0, los_asteroid_vectors[:]))

vectors_q1 = add_radian(vectors_q1)
vectors_q2 = add_radian(vectors_q2)
vectors_q3 = add_radian(vectors_q3)
vectors_q4 = add_radian(vectors_q4)
    
print(vectors_q1)
print()
print(vectors_q2)
print()
print(vectors_q3)
print()
print(vectors_q4)

[(0, 23, 90.0), (1, 23, 87.510447), (4, 23, 80.134193), (5, 23, 77.735226), (9, 23, 68.629378), (1, 22, 87.397438), (2, 22, 84.805571), (4, 22, 79.695154), (9, 22, 67.750976), (2, 21, 84.559668), (5, 21, 76.607502), (6, 21, 74.054604), (7, 21, 71.565051), (8, 21, 69.145542), (1, 20, 87.137595), (3, 20, 81.469234), (9, 20, 65.772255), (1, 19, 86.987212), (8, 19, 67.166346), (1, 18, 86.82017), (2, 18, 83.659808), (3, 18, 80.537678), (4, 18, 77.471192), (9, 18, 63.434949), (4, 17, 76.75948), (5, 17, 73.61046), (7, 17, 67.619865), (8, 17, 64.798876), (2, 16, 82.874984), (5, 16, 72.645975), (6, 16, 69.443955), (7, 16, 66.370622), (9, 16, 60.642246), (1, 15, 86.185925), (2, 15, 82.405357), (3, 15, 78.690068), (7, 15, 64.983107), (8, 15, 61.927513), (9, 15, 59.036243), (1, 14, 85.914383), (2, 14, 81.869898), (3, 14, 77.905243), (3, 13, 77.005383), (5, 13, 68.962489), (6, 13, 65.224859), (7, 13, 61.699244), (8, 13, 58.392498), (3, 12, 75.963757), (9, 12, 53.130102), (5, 11, 65.556045), (9, 11,

In [188]:
# iterate over first 200 asteroids in clockwise fashion, return the coordinates of the 200th asteroid
next_closest_vector = None

for _ in range(200):
    # 90 deg -> 0 deg
    if vectors_q1: # there are still asteroids in q1. So the order to scan them are from 1.570796 radian -> 0 radian
        current_distance = math.inf
        for vector in vectors_q1:
            if 90 - vector[2] < current_distance:
                current_distance = 90 - vector[2]
                next_closest_vector = vector
                print(next_closest_vector)
                
        vectors_q1.remove(next_closest_vector)
    
    # -0 deg -> -90 deg
    elif vectors_q2:
        current_distance = -math.inf
        for vector in vectors_q2:
            if vector[2] > current_distance:
                current_distance = vector[2]
                next_closest_vector = vector
                print(next_closest_vector)
                
        vectors_q2.remove(next_closest_vector)
    
    # 90 deg -> 0 deg
    elif vectors_q3:
        current_distance = math.inf
        for vector in vectors_q3:
            if 90 - vector[2] < current_distance:
                current_distance = 90 - vector[2]
                next_closest_vector = vector
                print(next_closest_vector)
                
        vectors_q3.remove(next_closest_vector)
        
     # -0 deg -> -90 deg
    elif vectors_q4:
        current_distance = -math.inf
        for vector in vectors_q4:
            if vector[2] > current_distance:
                current_distance = vector[2]
                next_closest_vector = vector
                print(next_closest_vector)
                
        vectors_q4.remove(next_closest_vector)
        
print(next_closest_vector)

(0, 23, 90.0)
(1, 23, 87.510447)
(4, 23, 80.134193)
(1, 22, 87.397438)
(4, 23, 80.134193)
(2, 22, 84.805571)
(1, 20, 87.137595)
(4, 23, 80.134193)
(2, 22, 84.805571)
(1, 19, 86.987212)
(4, 23, 80.134193)
(2, 22, 84.805571)
(1, 18, 86.82017)
(4, 23, 80.134193)
(2, 22, 84.805571)
(1, 15, 86.185925)
(4, 23, 80.134193)
(2, 22, 84.805571)
(1, 14, 85.914383)
(4, 23, 80.134193)
(2, 22, 84.805571)
(4, 23, 80.134193)
(2, 21, 84.559668)
(4, 23, 80.134193)
(3, 20, 81.469234)
(2, 18, 83.659808)
(1, 10, 84.289407)
(4, 23, 80.134193)
(3, 20, 81.469234)
(2, 18, 83.659808)
(4, 23, 80.134193)
(3, 20, 81.469234)
(2, 16, 82.874984)
(4, 23, 80.134193)
(3, 20, 81.469234)
(2, 15, 82.405357)
(4, 23, 80.134193)
(3, 20, 81.469234)
(2, 14, 81.869898)
(4, 23, 80.134193)
(3, 20, 81.469234)
(4, 23, 80.134193)
(3, 18, 80.537678)
(4, 23, 80.134193)
(5, 23, 77.735226)
(4, 22, 79.695154)
(5, 23, 77.735226)
(3, 15, 78.690068)
(5, 23, 77.735226)
(3, 14, 77.905243)
(5, 23, 77.735226)
(9, 23, 68.629378)
(5, 21, 76.607502)

(-17, 6, -19.440035)
(-16, 5, -17.354025)
(-14, 4, -15.945396)
(-17, 3, -10.00798)
(-16, 2, -7.125016)
(-17, 1, -3.366461)
(-16, 0, -0.0)
(-16, 23, -55.175511)
(-17, 22, -52.30576)
(-17, 21, -51.009006)
(-17, 19, -48.17983)
(-17, 18, -46.636577)
(-17, 17, -45.0)
(-17, 16, -43.264295)
(-17, 15, -41.423666)
(-17, 14, -39.47246)
(-17, 13, -37.405357)
(-17, 12, -35.217593)
(-17, 11, -32.905243)
(-17, 9, -27.897271)
(-17, 8, -25.201124)
(-17, 7, -22.380135)
(-17, 6, -19.440035)
(-16, 5, -17.354025)
(-14, 4, -15.945396)
(-17, 3, -10.00798)
(-16, 2, -7.125016)
(-17, 1, -3.366461)
(-16, 23, -55.175511)
(-17, 22, -52.30576)
(-17, 21, -51.009006)
(-17, 19, -48.17983)
(-17, 18, -46.636577)
(-17, 17, -45.0)
(-17, 16, -43.264295)
(-17, 15, -41.423666)
(-17, 14, -39.47246)
(-17, 13, -37.405357)
(-17, 12, -35.217593)
(-17, 11, -32.905243)
(-17, 9, -27.897271)
(-17, 8, -25.201124)
(-17, 7, -22.380135)
(-17, 6, -19.440035)
(-16, 5, -17.354025)
(-14, 4, -15.945396)
(-17, 3, -10.00798)
(-16, 2, -7.125016

(-15, 6, -21.801409)
(-13, 5, -21.037511)
(-16, 23, -55.175511)
(-17, 22, -52.30576)
(-17, 21, -51.009006)
(-17, 19, -48.17983)
(-17, 18, -46.636577)
(-17, 17, -45.0)
(-17, 16, -43.264295)
(-17, 15, -41.423666)
(-17, 14, -39.47246)
(-17, 13, -37.405357)
(-17, 12, -35.217593)
(-17, 11, -32.905243)
(-17, 9, -27.897271)
(-17, 8, -25.201124)
(-17, 7, -22.380135)
(-15, 6, -21.801409)
(-16, 23, -55.175511)
(-17, 22, -52.30576)
(-17, 21, -51.009006)
(-17, 19, -48.17983)
(-17, 18, -46.636577)
(-17, 17, -45.0)
(-17, 16, -43.264295)
(-17, 15, -41.423666)
(-17, 14, -39.47246)
(-17, 13, -37.405357)
(-17, 12, -35.217593)
(-17, 11, -32.905243)
(-17, 9, -27.897271)
(-17, 8, -25.201124)
(-17, 7, -22.380135)
(-16, 23, -55.175511)
(-17, 22, -52.30576)
(-17, 21, -51.009006)
(-17, 19, -48.17983)
(-17, 18, -46.636577)
(-17, 17, -45.0)
(-17, 16, -43.264295)
(-17, 15, -41.423666)
(-17, 14, -39.47246)
(-17, 13, -37.405357)
(-17, 12, -35.217593)
(-17, 11, -32.905243)
(-17, 9, -27.897271)
(-17, 8, -25.201124)
(

(-17, 21, -51.009006)
(-17, 19, -48.17983)
(-17, 18, -46.636577)
(-17, 17, -45.0)
(-17, 16, -43.264295)
(-17, 15, -41.423666)
(-14, 12, -40.601295)
(-16, 23, -55.175511)
(-17, 22, -52.30576)
(-17, 21, -51.009006)
(-17, 19, -48.17983)
(-17, 18, -46.636577)
(-17, 17, -45.0)
(-17, 16, -43.264295)
(-17, 15, -41.423666)
(-8, 7, -41.185925)
(-16, 23, -55.175511)
(-17, 22, -52.30576)
(-17, 21, -51.009006)
(-17, 19, -48.17983)
(-17, 18, -46.636577)
(-17, 17, -45.0)
(-17, 16, -43.264295)
(-17, 15, -41.423666)
(-16, 23, -55.175511)
(-17, 22, -52.30576)
(-17, 21, -51.009006)
(-17, 19, -48.17983)
(-17, 18, -46.636577)
(-17, 17, -45.0)
(-17, 16, -43.264295)
(-12, 11, -42.510447)
(-11, 10, -42.273689)
(-9, 8, -41.633539)
(-16, 23, -55.175511)
(-17, 22, -52.30576)
(-17, 21, -51.009006)
(-17, 19, -48.17983)
(-17, 18, -46.636577)
(-17, 17, -45.0)
(-17, 16, -43.264295)
(-12, 11, -42.510447)
(-11, 10, -42.273689)
(-16, 23, -55.175511)
(-17, 22, -52.30576)
(-17, 21, -51.009006)
(-17, 19, -48.17983)
(-17, 

In [184]:
(-11 * 100) + 14

-1086